# Milestone 1

## Import

In [1]:
import json
import os
import requests
import pandas as pd
import zipfile

## Download the Data

### Download from API

In [3]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
output_dir = "../data/"

In [5]:
resp = requests.get(url)

file_list = json.loads(resp.content)['files']

Note: The cell below takes around 1.5 minutes to run. 

In [7]:
%%time
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
for file in file_list:
    if file['name'] != "data.zip":
        continue
    output_name = os.path.join(output_dir, file['name'])
    if os.path.exists(output_name):
        print(f"{file['name']} already exists!")
    else:
        print(f"Downloading {file['name']} to {output_dir}...")
        file_resp = requests.get(file['download_url'])
        with open(output_name, "wb") as f:
            for chunk in file_resp.iter_content(chunk_size=1024): 
                if chunk:
                    f.write(chunk)
        print("Download finished")

data.zip already exists!
Wall time: 1e+03 µs


### Unzip data

In [8]:
raw_csv_dir = os.path.join(output_dir, "raw")

In [9]:
%%time
with zipfile.ZipFile(output_name, "r") as zf:
    zf.extractall(raw_csv_dir)

Wall time: 33.4 s


## Combine CSV Files

In [10]:
dir_files = os.listdir(raw_csv_dir)
output_df = pd.DataFrame()

In [18]:
%%time
# Exclude MACOSX_ & observed_daily_rainfall_SYD.csv
for fname in dir_files:
    if fname[-8:] != "_NSW.csv":
        continue

    model_name = fname.split('_')[0]

    df = pd.read_csv(os.path.join(raw_csv_dir, fname), index_col=0)
    df['model'] = model_name
    output_df = pd.concat([output_df, df])
    del df

output_df.to_csv(os.path.join(output_dir, "combined.csv"))

NameError: name 'output_df' is not defined

In [12]:
%%sh
du -sh ../data/combined.csv

5.7G	../data/combined.csv


In [13]:
output_df.shape

(62467843, 6)

In [14]:
output_df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


In [15]:
del output_df

### Comparison

| Team Member| Operating System | RAM        | Processor                        | Is SSD | CPU Time   | Wall Time |
|:----------:|:----------------:|:----------:|:--------------------------------:|:------:|:----------:|:---------:|
| James      | Big Sur 11.5     | 8 GB DDR3  | 1.4 GHz Quad-Core Intel Core i5  | Yes    | 7min 54s   | 8min 10s  |
| Kyle       |                  |            |                                  |        |            | 8min 15s  |
| LG         |                  |            |                                  |        |            |           |
| Philson    | Ubuntu 20.04     | 16 GB DDR4 | 1.8GHz Quad-Core Intel i7-8565U  | Yes    | 6min 39s   | 7min 06s  |

**Discussion:**
> Placeholder

## Load the Combined CSV and simple EDA

### Approach 1: Load Only Columns We want

Let's try `value_counts()` on `model` column. Let's compare how long it takes when we only load in `model` column vs all columns.

In [20]:
%%time
use_cols = ["model"]
df = pd.read_csv("../data/combined.csv", usecols=use_cols)
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
Wall time: 46.7 s


In [22]:
%%time
df = pd.read_csv("../data/combined.csv")
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
Wall time: 1min 14s


#### Comparison - Load Only Columns We Want

| Team Member| Operating System | RAM        | Processor        | Is SSD | CPU Time (model)   | Wall Time (model) |  CPU Time (all) | Wall Time (all) |
|:----------:|:----------------:|:----------:|:--------------------------------:|:------:|:----------:|:---------:|:--------:|:-------:|
| James      | Big Sur 11.5     | 8 GB DDR3  | 1.4 GHz Quad-Core Intel Core i5  | Yes    |            |     |      | 
| Kyle       | WIndows 11       | 32 GB      | 2.2 GHz Hexa-Core Intel i7-8750H | Yes    |            | 46.5s|     |1min 23s
| LG         |                  |            |                                  |        |            |     |      |
| Philson    | Ubuntu 20.04     | 16 GB DDR4 | 1.8GHz Quad-Core Intel i7-8565U  | Yes    |            |     |      |

**Discussion:**
> We can observe that when only `model` column is loaded, there is a significant reduction in Wall Time.

### Approach 2

In [28]:
%%time
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("../data/combined.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int32
Wall time: 1min 14s


#### Comparison - Load in Chunk

| Team Member| Operating System | RAM        | Processor        | Is SSD | CPU Time (chunk)   | Wall Time (chunk) |  CPU Time (all) | Wall Time (all) |
|:----------:|:----------------:|:----------:|:--------------------------------:|:------:|:----------:|:---------:|:--------:|:-------:|
| James      | Big Sur 11.5     | 8 GB DDR3  | 1.4 GHz Quad-Core Intel Core i5  | Yes    |            |     |      | 
| Kyle       | WIndows 11       | 32 GB      | 2.2 GHz Hexa-Core Intel i7-8750H | Yes    |            | 1min 14s|     |1min 23s
| LG         |                  |            |                                  |        |            |     |      |
| Philson    | Ubuntu 20.04     | 16 GB DDR4 | 1.8GHz Quad-Core Intel i7-8565U  | Yes    |            |     |      |

**Discussion:**
> We can observe that there is a reasonable reduction in Wall Time when we run `value_counts()` by loading in smaller chunks, 4 million rows each in our case.

## Simple EDA in R

### Instruction:
1. Pick an approach to transfer the dataframe from python to R.
- Parquet file
- Feather file
- Pandas exchange
- Arrow exchange
2. Discuss why you chose this approach over others.

In [12]:
%load_ext rpy2.ipython

In [13]:
%%R
library(dplyr)
library(arrow)

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: ‘arrow’


R[write to console]: The following object is masked from ‘package:utils’:

    timestamp




### 6.1 Using arrow table and pyra 

In [15]:
import pyarrow.dataset as ds
import pyarrow as pa
import pandas as pd
import pyarrow 
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra

In [16]:
%%time
# loading csv file to pyarrow dataset and then converting to pyarrow table
dataset = ds.dataset("../data/combined.csv", format="csv")
table = dataset.to_table()

CPU times: user 20.8 s, sys: 4.3 s, total: 25.1 s
Wall time: 23.1 s


In [17]:
%%time
# converting pyarrow table to rarrow table
r_table = pyra.converter.py2rpy(table)

CPU times: user 30.8 s, sys: 553 ms, total: 31.3 s
Wall time: 31.7 s


In [18]:
%%time
%%R -i r_table
# referenced lecture 2 note
# performing simple EDA to count the number of each model
start_time <- Sys.time()
library(dplyr)
counts <- r_table %>% collect() %>% count(model)
end_time <- Sys.time()

print(counts)
print(end_time - start_time)

# A tibble: 27 × 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# … with 17 more rows
Time difference of 9.245153 secs
CPU times: user 5.62 s, sys: 4.89 s, total: 10.5 s
Wall time: 9.42 s


### 6.2 Using feather

In [21]:
%%time
# creating feather file from above table
import pyarrow.feather as feather
feather.write_feather(table, '../data/combined.feather')

CPU times: user 5.31 s, sys: 16.5 s, total: 21.8 s
Wall time: 11.2 s


In [25]:
%%time
%%R
# transfering the feather file to R
r_table_feather <- arrow::read_feather("../data/combined.feather")

CPU times: user 4.77 s, sys: 7.12 s, total: 11.9 s
Wall time: 5.06 s


In [27]:
%%time
%%R
# performing the same EDA as above
start_time <- Sys.time()
library(dplyr)
counts <- r_table_feather %>% collect() %>% count(model)
end_time <- Sys.time()

print(counts)
print(end_time - start_time)

# A tibble: 27 × 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# … with 17 more rows
Time difference of 3.866908 secs
CPU times: user 1.6 s, sys: 1.67 s, total: 3.27 s
Wall time: 3.92 s


### Discussion
- We chose to experiment with Feather with the following reasons:  
- According to various sources, Feather is faster than Parquet to read into Arrow Table although not recommended for long-term data storage.
- Compared to vanila serialization/deserialization process as performed above, Feather indeed performed much faster.
- Using Feather file also saved the storage, as the file generated is only 1.1 GB